In [1]:
# Function: calculate and sum up land sales merged with generator data
# Date: 2021-7-25
# Output Dataset： land_dataset.xlsx

In [2]:
import numpy as np
import pandas as pd

In [3]:
dataset = pd.read_table('Z:/User/Documents/land/land2.txt', delimiter=',')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
dataset.columns

Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'JOIN_FID', '省份', '城市', '区县',
       '行业分公', '土地级公', '成交价公', '土地价公_1', '经度', '纬度', '约定交公', '约定开公', '约定竣公',
       '合同签公', 'chname', 'ename', 'gbcode', 'code'],
      dtype='object')

In [5]:
dataset2 = dataset.reindex(['省份','城市','合同签公','chname','gbcode'], axis=1)

In [6]:
dataset2 = dataset2[dataset2['合同签公'].notna()]

In [7]:
import re

In [8]:
def check_num(s):
    pattern = r'[^0-9.]+'
    pattern = re.compile(pattern)
    res = ''.join(pattern.findall(s)[:])
    return res

In [9]:
def check_letter(s):
    pattern = r'[^a-zA-Z]'
    pattern = re.compile(pattern)
    res = pattern.findall(s)
    if res is None:
        

SyntaxError: unexpected EOF while parsing (<ipython-input-9-506d0f313afd>, line 6)

In [10]:
dataset3 = dataset2[dataset2.index<=1103104]

In [11]:
dataset3.index = pd.to_datetime(dataset3['合同签公'])

In [12]:
dataset3['year'] = dataset3.to_period(freq='A').index

<ipython-input-12-a754d5acc132>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset3['year'] = dataset3.to_period(freq='A').index


In [13]:
dataset3['dummy'] = 1

<ipython-input-13-f997ef3bffd2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset3['dummy'] = 1


In [14]:
dataset4 = dataset3.groupby(['gbcode','year'])['合同签公'].count().reset_index()

In [18]:
cities = dataset3[['gbcode','省份']].reset_index()[['gbcode','省份']]

In [19]:
cities

,gbcode,省份
0,110103.0,北京市
1,110101.0,北京市
2,110105.0,北京市
3,110101.0,北京市
4,110101.0,北京市
...,...,...
1103100,430581.0,湖南省
1103101,430521.0,湖南省
1103102,430521.0,湖南省
1103103,430521.0,湖南省


In [20]:
dataset4.merge(cities, how = 'inner')

,gbcode,year,合同签公,省份
0,110101.0,2002,7,北京市
1,110101.0,2002,7,北京市
2,110101.0,2002,7,北京市
3,110101.0,2002,7,北京市
4,110101.0,2002,7,北京市
...,...,...,...,...
16042932,659004.0,2018,2,新疆建设兵团
16042933,659004.0,2018,2,新疆建设兵团
16042934,659004.0,2018,2,新疆建设兵团
16042935,659004.0,2018,2,新疆建设兵团


In [21]:
dataset4['year'] = dataset4['year'].astype('str').astype('int64')

In [22]:
capacity = pd.read_excel('Z:/User/Documents/capacity3.xlsx')

In [23]:
res_data = capacity.merge(dataset4, how='inner').sort_values(by='year')

In [24]:
res_data

,gbcode,year,total_capacity,province,合同签公
549,130124,2000,0,Hebei,2
21930,500103,2000,0,,2
12348,350825,2000,0,,3
22442,510106,2000,0,,10
22411,510104,2000,0,,2
...,...,...,...,...,...
3587,150123,2016,0,,8
11147,341282,2016,0,,9
22027,500109,2016,0,,30
11159,341302,2016,1735,Anhui,128


In [30]:
res_data.to_excel('land_dataset2.xlsx', index=False)